# Урок 3. Системы управления базами данных MongoDB и SQLite в Python

1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.

3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [1]:
!pip install pymongo

In [2]:
from pymongo import MongoClient
import json

In [3]:
with open('hh_Python.json') as f:
    file_data = json.load(f)

client = MongoClient('localhost', 27017) #создаем клиент
db = client['vacancy_db'] #создаем БД, [] - имя БД
hh_vacancies = db.vacancies #создаем привязку к коллекции vacancy в БД

hh_vacancies.insert_many(file_data)

# поиск по БД (зарплата)
def find_salary():
    x = int(input('Введите заработную плату для поиска: '))
    for vacancy in hh_vacancies.find({'$or':[
                                      {'salary_min': {'$gt': x}}, {'salary_max': {'$gt': x}}
                                     ]},
                                      {'_id':0, 'employer': 1, 'name': 1, 'salary_min': 1, 'salary_max': 1}):
        print(vacancy)

find_salary()

# запись в БД новых вакансий

def add_to_db(data):
    def is_existed_vacancy(vacancy_data):
        a=hh_vacancies.find_one(vacancy_data)
        if a:
            return True
        return False
    n, o =0, 0
    for i in data:
        existed_vacancy = is_existed_vacancy(i)
        if not existed_vacancy:
            n+=1
            hh_vacancies.insert_one(i)
        else:
            o+-1
    print(f'Got {len(data)} vacancies, {n} new, {o} are not written in db')

Введите заработную плату для поиска: 200000
{'name': 'Технический директор / Архитектор ПО (удаленно)', 'employer': ' ZennoLab', 'salary_min': 250000, 'salary_max': 300000}
{'name': 'Quantitative developer (HFT developer)', 'employer': ' Cindicator', 'salary_min': None, 'salary_max': 400000}
{'name': 'DevOps Engineer', 'employer': ' ZAVOD Games', 'salary_min': 170000, 'salary_max': 250000}
{'name': 'Backend-разработчик Python', 'employer': 'ООО Мирамедикс', 'salary_min': 170000, 'salary_max': 230000}
{'name': 'Python developer (Django)', 'employer': 'ООО Айдвелл', 'salary_min': 150000, 'salary_max': 220000}
{'name': 'Python QA Automation', 'employer': 'ООО Кузница Кадров', 'salary_min': 120000, 'salary_max': 240000}
{'name': 'Технический директор / Архитектор ПО (удаленно)', 'employer': ' ZennoLab', 'salary_min': 250000, 'salary_max': 300000}
{'name': 'Quantitative developer (HFT developer)', 'employer': ' Cindicator', 'salary_min': None, 'salary_max': 400000}
{'name': 'DevOps Engineer